# **Credit Card Fraud Detection**

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier

In [2]:
cc_data = pd.read_csv('data/creditcard.csv')
cc_data.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [3]:
cc_data.shape

(284807, 31)

In [4]:
cc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [5]:
cc_data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [6]:
cc_data.isna().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [7]:
cc_data['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [8]:
legit = cc_data[cc_data['Class'] == 0]
fraud = cc_data[cc_data['Class'] == 1]

print(legit.shape)
print(fraud.shape)
print(legit['Amount'].describe())
print(fraud['Amount'].describe())

(284315, 31)
(492, 31)
count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64
count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64


In [9]:
cc_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


### Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions --> 492

To Avoid **Underfitting on the Minority Class** and **Risk of Overfitting on Small Minority Data**

In [10]:
legit_sample = legit.sample(n=492)
new_dataset = pd.concat([legit_sample, fraud], axis=0)
new_dataset.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
133204,80288.0,-1.598463,1.559066,-0.720122,1.051907,-0.412675,-1.004369,0.192708,0.560359,-0.216191,...,0.254704,0.690786,0.270203,0.382932,-0.887267,-0.516098,-0.933959,-0.399342,2.99,0
201468,133896.0,-1.718164,-0.341942,0.067361,-0.846432,0.741112,-0.212879,2.216020,-0.205570,-0.371936,...,0.006636,-0.665379,0.503539,0.345234,1.095375,-0.083115,-0.144181,0.118804,383.72,0
213506,139283.0,-0.235945,1.367876,1.389625,4.756042,0.421671,0.831241,0.470093,0.042590,-1.279555,...,-0.219200,-0.342885,0.087573,-0.029528,-0.845149,0.091890,-0.015161,0.222929,8.34,0
88406,62127.0,-1.096215,0.330963,2.422721,-0.985957,-0.403855,-0.382953,0.432195,-0.152846,0.972608,...,-0.237980,-0.235702,-0.093064,0.469092,-0.099258,0.747308,-0.156432,-0.034544,30.58,0
168418,119219.0,2.215150,-0.833440,-1.453347,-1.059423,-0.516494,-0.888584,-0.552711,-0.144950,-0.518990,...,0.443714,1.219463,-0.030877,-0.379725,0.150390,0.081246,-0.048998,-0.084945,3.90,0
84512,60338.0,-0.623624,1.163065,1.032886,-0.232199,0.329547,-0.026792,0.480410,0.275436,-0.360142,...,-0.243656,-0.590981,-0.024207,-0.378727,-0.188332,0.111033,0.361698,0.139306,3.87,0
206907,136433.0,2.039024,0.091137,-1.799121,0.522730,0.121656,-1.460889,0.221442,-0.332782,0.753892,...,0.182914,0.623206,-0.039160,-0.201257,0.233175,-0.092435,-0.011944,-0.032285,17.90,0
86018,61048.0,1.212267,-0.293961,-0.658980,0.309513,1.770024,4.043145,-0.883993,1.027177,0.808259,...,-0.162437,-0.286265,-0.142694,1.027881,0.850641,-0.288489,0.059450,0.019898,13.01,0
58728,48520.0,-0.384783,1.050227,1.302912,-0.049947,0.041879,-0.661910,0.590924,0.096096,-0.594117,...,-0.216067,-0.610377,-0.018880,0.266732,-0.203542,0.055870,0.238630,0.087281,5.16,0
132330,79945.0,-1.445561,0.112434,1.240895,1.172598,0.019078,0.680232,-0.283754,0.721065,0.278888,...,0.086891,0.290325,-0.128348,-0.757021,0.135988,-0.144394,-0.052086,0.025150,64.86,0


In [11]:
new_dataset['Class'].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [12]:
X = new_dataset.drop(columns='Class', axis=1)
y = new_dataset['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [13]:
X_train.shape

(787, 30)

### **Logistic Regression**

In [14]:
param_grid = {
    'lr__penalty': ['elasticnet'], 
    'lr__C': [1],                   
    'lr__solver': ['saga'],        
    'lr__max_iter': [1000],                    
    'lr__l1_ratio': [ 0.7]    
}

pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),  
    ('lr', LogisticRegression())  
])

grid_search_lr = GridSearchCV(estimator=pipeline_lr, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_lr.fit(X_train, y_train) 

prediction_data_lr = grid_search_lr.predict(X_test)
train_accuracy_lr = accuracy_score(y_train, grid_search_lr.predict(X_train))
test_accuracy_lr = accuracy_score(y_test, prediction_data_lr)

print("Best parameters found: ", grid_search_lr.best_params_)
print("Best cross-validation accuracy: {:.3f}".format(grid_search_lr.best_score_))
print("Training accuracy: {:.3f}".format(train_accuracy_lr))
print("Test accuracy: {:.3f}".format(test_accuracy_lr))

if train_accuracy_lr < test_accuracy_lr and test_accuracy_lr < grid_search_lr.best_score_:
    print("Underfitting: The model may be too simple for the data.")
elif train_accuracy_lr > test_accuracy_lr:
    print("Overfitting: The model may be overfitting to the training data.")
else:
    print("Good fit: The model seems to generalize well.")

Best parameters found:  {'lr__C': 1, 'lr__l1_ratio': 0.7, 'lr__max_iter': 1000, 'lr__penalty': 'elasticnet', 'lr__solver': 'saga'}
Best cross-validation accuracy: 0.930
Training accuracy: 0.940
Test accuracy: 0.975
Good fit: The model seems to generalize well.


### **Random Forest**

In [15]:
param_grid_rf = {
    'n_estimators': [200, 300],
    'max_depth': [10,],
    'min_samples_split': [10,15,20],
    'min_samples_leaf': [4,6,8,10],
    'max_features': ['sqrt'],     
    'bootstrap': [ False]  
}

model_rf = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(model_rf, param_grid_rf, cv=3, n_jobs=-1, verbose=1, scoring='accuracy')
grid_search_rf.fit(X_train, y_train)

prediction_data_rf = grid_search_rf.predict(X_test)
train_accuracy_rf = accuracy_score(y_train, grid_search_lr.predict(X_train))
test_accuracy_rf = accuracy_score(y_test, prediction_data_rf)

print("Best parameters found: ", grid_search_rf.best_params_)
print("Best cross-validation accuracy: {:.3f}".format(grid_search_rf.best_score_))
print("Training accuracy: {:.3f}".format(train_accuracy_rf))
print("Test accuracy: {:.3f}".format(test_accuracy_rf))

if train_accuracy_rf < test_accuracy_rf and test_accuracy_rf < grid_search_lr.best_score_:
    print("Underfitting: The model may be too simple for the data.")
elif train_accuracy_rf > test_accuracy_rf:
    print("Overfitting: The model may be overfitting to the training data.")
else:
    print("Good fit: The model seems to generalize well.")

Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best parameters found:  {'bootstrap': False, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 15, 'n_estimators': 300}
Best cross-validation accuracy: 0.933
Training accuracy: 0.940
Test accuracy: 0.959
Good fit: The model seems to generalize well.


### **Gradient Boosting**

In [16]:
param_grid = {
    'n_estimators': [100],     
    'learning_rate': [ 0.1, 0.2],  
    'max_depth': [3, 4],   
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1], 
    'subsample': [0.8, 0.9],  
    'max_features': ['sqrt']  
}

grid_search_gb = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid, cv=3, n_jobs=-1, verbose=1, scoring='accuracy')
grid_search_gb.fit(X_train, y_train)
predictions_gb = grid_search_gb.predict(X_test)

train_accuracy_gb = accuracy_score(y_train, grid_search_gb.predict(X_train))
test_accuracy_gb = accuracy_score(y_test, predictions_gb)

print("Best parameters found: ", grid_search_gb.best_params_)
print("Best cross-validation accuracy: {:.3f}".format(grid_search_gb.best_score_))
print("Training accuracy: {:.3f}".format(train_accuracy_gb))
print("Test accuracy: {:.3f}".format(test_accuracy_gb))

if train_accuracy_gb < test_accuracy_gb and test_accuracy_gb < grid_search_lr.best_score_:
    print("Underfitting: The model may be too simple for the data.")
elif train_accuracy_gb > test_accuracy_gb:
    print("Overfitting: The model may be overfitting to the training data.")
else:
    print("Good fit: The model seems to generalize well.")

Fitting 3 folds for each of 2916 candidates, totalling 8748 fits
Best parameters found:  {'learning_rate': 0.1, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100, 'subsample': 0.9}
Best cross-validation accuracy: 0.938
Training accuracy: 1.000
Test accuracy: 0.975
Overfitting: The model may be overfitting to the training data.


### **XGBoosting**

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],  
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6],   
    'min_child_weight': [1, 2, 3], 
    'subsample': [0.6, 0.8, 1.0], 
    'colsample_bytree': [0.6, 0.8, 1.0], 
    'gamma': [0, 0.1, 0.2], 
    'reg_alpha': [0, 0.1, 1], 
    'reg_lambda': [1, 1.5, 2] 
}

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search_xgb.fit(X_train, y_train)
predictions_xgb = grid_search_gb.predict(X_test)

train_accuracy_xgb = accuracy_score(y_train, grid_search_xgb.predict(X_train))
test_accuracy_xgb = accuracy_score(y_test, predictions_xgb)

print("Best parameters found: ", grid_search_xgb.best_params_)
print("Best cross-validation accuracy: {:.3f}".format(grid_search_xgb.best_score_))
print("Training accuracy: {:.3f}".format(train_accuracy_xgb))
print("Test accuracy: {:.3f}".format(test_accuracy_xgb))

if train_accuracy_xgb < test_accuracy_xgb and test_accuracy_xgb < grid_search_lr.best_score_:
    print("Underfitting: The model may be too simple for the data.")
elif train_accuracy_xgb > test_accuracy_xgb:
    print("Overfitting: The model may be overfitting to the training data.")
else:
    print("Good fit: The model seems to generalize well.")